首先，尝试OpenAI的API请求：

In [1]:
from openai import OpenAI
api_key = "你的API密钥"
client = OpenAI(api_key=api_key)


completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    temperature=0.7,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "美国总统是谁？"
        }
    ]
)


print(completion.choices[0].message.content)

2021年，美国总统是乔·拜登（Joe Biden）。


回答：2021年，美国总统是乔·拜登（Joe Biden）。则说明API请求成功。
接下来，尝试使用qa.txt文件实现一个简单的RAG。这里，我们使用Chroma数据库进行向量存储。

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader

import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

def load_data(file_path):
    """加载文本数据"""
    loader = TextLoader(file_path, encoding='utf-8')
    return loader.load()


def split_text(data):
    """分割文本为更小的块"""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)
    return text_splitter.split_documents(data)


def create_vectorstore(splits):
    persist_directory = r"D:\good_good_study\111cs_study\Python\LLM-langchain\mydb.db"  # 指定存储文件的路径
    vectorstore = Chroma.from_documents(
        documents=splits, embedding=OpenAIEmbeddings(), persist_directory=persist_directory
    )
    """创建 Chroma 向量存储"""
    return vectorstore


def answer_question(question, qa_chain):
    """获取问题的答案"""
    try:
        result = qa_chain({"query": question})
        return result['result']
    except Exception as e:
        return f"发生错误：{e}"


def run():
    """主函数，处理用户输入"""
    # 加载并处理数据
    data = load_data('qa.txt')
    all_splits = split_text(data)
    vectorstore = create_vectorstore(all_splits)

    # 初始化问答链
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())

    # 用户提问循环
    while True:
        # print("run")
        query = input("请输入问题，例如：弦丝画制作的活动时长是多少？\n")
        if query.lower() == "end":
            print("程序结束！")
            break

        print(f"问题：{query}")
        answer = qa_chain.invoke({"query": query})
        print(f"回答：{answer['result']}")



if __name__ == "__main__":
    run()

下面逐段解释以上代码：

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

这段代码获取OpenAI的API密钥，以便后续代码调用OpenAI的大模型。需要注意正确设置环境变量OPENAI_API_KEY，建议在Jupyter Notebook或Colab notebook中运行，方便设置环境变量。
在Jupyter Notebook或Colab notebook中，可以通过以下代码段设置环境变量。

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = '你的OpenAI_API_KEY'

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)